In [25]:
!pip install crewai langchain-google-genai langchain-community langchain

n /usr/local/lib/python3.11/dist-packages (from onnxruntime==1.22.0->crewai) (24.2)
=0.5.23->crewai) (1.73.1)


In [26]:
from crewai import Agent, Task, Crew
from langchain_community.chat_models import ChatLiteLLM


In [ ]:
api_key = "UPLOAD_YOUR_GEMINI_API_KEY_HERE"

llm = ChatLiteLLM(
    model = "gemini/gemini-1.5-flash",
    temerature = 0.3,
    api_key = api_key
)

In [28]:
text = """"
      RCB Finally Breaks the Curse to Lift IPL 2025 Trophy in a Thrilling Final
      In a historic night at the Narendra Modi Stadium in Ahmedabad, Royal Challengers Bengaluru (RCB) finally laid their hands on the elusive IPL trophy after 18 long years, defeating Punjab Kings (PBKS) by six runs in a nail-biting TATA IPL 2025 final. Batting first after winning the toss, RCB posted a competitive total of 190/9, thanks to Virat Kohli’s composed 43 off 35 balls, supported by crucial cameos from Mayank Agarwal, Rajat Patidar, and a fiery 17 off 9 by Romario Shepherd in the death overs. Punjab’s bowlers, particularly Arshdeep Singh and Kyle Jamieson, kept the pressure high by sharing six wickets between them, but RCB’s resilience with the bat ensured a defendable score. Chasing 191 for victory, PBKS started steadily with Prabhsimran Singh and Priyansh Arya adding valuable runs up front. However, RCB’s bowlers tightened the screws with disciplined spells—Krunal Pandya’s 2 for 17 being the game-changer in the middle overs. The game took a dramatic turn when Shashank Singh unleashed a blistering counterattack, scoring an unbeaten 61, smashing six sixes in the final stretch. Despite his heroics, Punjab fell agonizingly short, ending at 184/7. The stadium erupted as RCB secured their first-ever IPL title, with emotional scenes unfolding as Kohli hugged long-time friends AB de Villiers and Chris Gayle, chanting the iconic “Ee Sala Cup Namdu.” The fairytale win, however, was shadowed by tragedy the following day, when a massive crowd surge during RCB’s victory celebration in Bengaluru led to a stampede outside Vidhana Soudha, claiming 11 lives and injuring dozens. While the cricketing world toasted RCB’s redemption story, the incident served as a sobering reminder of the responsibilities that come with such triumphs. Nonetheless, the 2025 final will be remembered for its intensity, emotional depth, and the end of RCB’s long-standing wait for IPL glory.
    """

# agent 1
news_extractor = Agent(
    role ="news extractor",
    goal = "evaluate the text and give the news report",
    backstory = "Expert in report writing",
    verbose = True,
    llm = llm
)

# agent 2
statistical_analyst = Agent(
    role = "statistical analyst",
    goal = "Evaluate the match and give the statistical report",
    backstory = "Expert in statistical analyis",
    verbose = True,
    llm = llm
)
# agent 3
fan_voice = Agent(
    role = "fan",
    goal = "Observe the match and give the commentry as a fan both for the positive and negative way",
    backstory = "You know the match rolues and regulations and you are a big fan of one team ",
    verbose = True,
    llm = llm
)
#agent 4
match_report = Agent(
    role = "Report writer",
    goal = "Create a Report for the match and give in the format of pdf",
    backstory = "Expert in creating Report",
    verbose = True,
    llm = llm
)

In [29]:
#task 1
extractor_task = Task(
    description = f"Check the following text and give the news according to that \n\n {text}",
    agent = news_extractor,
    expected_output= "A short description for the given task"
)
#task 2
statistics_task = Task(
    description="""
      Analyze the given context from that give the statistical report
    """,
    agent = statistical_analyst,
    context = [extractor_task],
    expected_output="give the statistical report"
)
#task 3
fan_task = Task(
    description=f"""
      Alayze the match and give the feedback as a fan for every match {text}
    """,
    agent = fan_voice,
    expected_output="a short content as a fan"

)
#task 4
report_task = Task(
    description = "Analyze the match and give the proper report for that based on the content that provied , format PDF",
    agent = match_report,
    context = [extractor_task, statistics_task, fan_task],
    expected_output= "give the entire report in the format of PDF"
)

In [31]:
crew = Crew(
    agents = [news_extractor, statistical_analyst,fan_voice,match_report],
    tasks = [extractor_task, statistics_task,fan_task,report_task],
    verbose = True
)

In [ ]:
# result = crew.kickoff()  
# print(result)

# uncomment the above line to run the crew and get the result